# Assignment II : Multli-layer Feed Forward Neural Network

Mohana Palaka

I.Mtech

15MCME01

In [53]:
import numpy as np
import math
import random
from mnist import MNIST
from sklearn.preprocessing import normalize

In [54]:
mndata = MNIST('./data') #Path for where the training and test data are

train_img, train_lbl = mndata.load_training() #Loading the training data
test_img, test_lbl = mndata.load_testing() #Loading the test data

#index = random.randrange(0, len(train_img))  
#print(mndata.display(train_img[0]))
#print(train_lbl[0])

The Multi-Layer Feed Forward Network Class :

In [98]:
class Net :
    
    def __init__(self) :
        
        self.h = 20       #initializing the number of hidden units (chosen arbitrarily)
        self.inp = 28*28  #Initializing the number of input nodes (Each input image is 28x28 and each pixel will be taken as an input)
        self.out = 10     #Initializing the number of output nodes
        
        self.i2h_weights = 1/28;   #we initialize the weight vector btw [-1/28,1/28]
        self.h2o_weights = 1/math.sqrt(self.h);   #we initialize the weight vector btw [+/-1/sqrt(h)]

        
        self.V = np.ones((self.inp, self.h))    #V, which will be our input matrix from input to hidden layer (the +1 is the bias)
        #self.W = np.ones((h+1, out))    #W, whcih will be our input matrix from hidden to output layer (the +1 is the bias)
        
        self.i_to_h_weights = 2*self.i2h_weights * np.random.random((self.h, self.inp)) - self.i2h_weights   #weights from input to hidden layer
        self.h_to_o_weights = 2*self.h2o_weights * np.random.random((self.out, self.h)) - self.h2o_weights    #Weights form hidden to output layer
         
        mndata = MNIST('./data') #Path for where the training and test data are

        self.train_img, self.train_lbl = mndata.load_training()  #Loading the training data (images and labels/correct outputs)
        self.test_img, self.test_lbl = mndata.load_testing()     #Loading the test data (images and labels/correct outputs)
        self.yd = np.zeros((10, 1))
        
        self.eta = 0.5     #Initializing the learning rate
        self.p_max = 60000
        self.err_max = 10
        
    
    def feed_forward(self, inputs, weights):  #feed-forward; sending the inputs from left to right and computing the outputs with no feedback
        
        z = weights.transpose() @ inputs  #weighted sum of the layer
        y = sig_fn(z)   #inputting the weighted sum into the sigmoid function

        return(y)
    
    
    def mse(self, n, y, yd):  #mean squared error
        
        err = 0
        
        for i in range(n):
                
                err = err + (1/2) * (y[i] - yd[i])**2
                
        err = err/n
        
        return(err)
        
        
    def batch_update(self):   #takes an array of all the inputs, and all their corresponding labels
        
        ep = 0 #Initializing the epoch increment variable
        err = 0
        
        while (err < self.err_max):
            
            ep = ep + 1  #Incrementing the number of epochs that the model has seen
            p = 0      #Pattern increment variable initialization 
            del_w = 0  #Initialization of change in weight vector from hidden to output layer
            del_v = 0  #Initialization of change in weight vector from input to hidden layer
            
            while (p <= self.p_max):
                
                curr_input = np.row_stack(train_img[p])   #converting the current training sample image, (which is a list) into a column matrix
                curr_input = curr_input/255     #scaling the inputs down to [0,1]
                
                yd = np.zeros((10, 1))   #initializing the correct output vector to zeros
                lbl = self.train_lbl[p]   #finding the correct output label of the current input vector
                yd[lbl] = 1  #assigning '1' to the output node corresponding to the correct label
                
                y_int = np.zeros((20, 1)) #initializing the intermediate output (output of hidden layer)
                y_output = np.zeros((10, 1))  #initializing the final outputs
                
                for i in range(self.h):  #Calculating all the hidden layer outputs (i took 20 hidden nodes)
                    
                    weights = np.row_stack(self.i_to_h_weights[i, :])   #the weights of the current inputs, to the i-th hidden node 
                    y_int[i][0] = self.feed_forward(curr_input, weights) #calculating the output of the hidden layer at the ith hidden node and storing it in a column matrix 'y intermediate'
                    #print("hidden y:", y_int[i, 0])
                    
                for i in range(self.out):  #calculating all the output layer outputs with the calculated hidden layer outputs(y intermediate)
                    
                    weights = np.row_stack(self.h_to_o_weights[i, :])  #weights of input to i-th output node
                    y_output[i] = self.feed_forward(y_int, weights)  #final output of the i-th output node
                
                pat_err = self.mse(self.out, y_output, yd)  #error of the current pattern
                
                #gradient desecent weight updation/back-propogation for hidden-to-input weights :
                for i in range(self.out):
                    
                    s = (y_output[i] - yd[i]) * y_output[i] * (1 - y_output[i])  #sensitivity for hidden to output weight updation
                    
                    for j in range(self.h):
                        
                        delw_h2o = (-self.eta) * s * y_int[j][0]  #delta rule
                        self.h_to_o_weights[i, j] = self.h_to_o_weights[i, j] + delw_h2o  #weight updation
                
                #gradient descent weight updation/back-propogation for input-to-hidden weights :
                for i in range(self.h):
                    
                    e = 0
                    
                    for j in range(self.out):
                        e = e + (y_output[j] - yd[j]) * y_output[j] * (1 - y_output[j]) * self.h_to_o_weights[j, i]  #summation of 
                        
                    s = e * y_int[i][0] * (1- y_int[i][0])  #sensitivity of input to hidden weights
                    #print(s)
                    for k in range(self.inp):
                        
                        delw_i2h = (-self.eta) * s * curr_input[k][0]  #delta rule
                        self.i_to_h_weights[i, k] = self.i_to_h_weights[i, k] + delw_i2h  #weight updation
                
                #print("i to h weights :", self.i_to_h_weights)
                #print("h to o weights :", self.h_to_o_weights)
                #print("Y :", y_output)
                #print("Yd :", yd)
                print("pattern err:", pat_err)
                err = err + pat_err  #error across all patterns
                p = p + 1  #incrementing the number of patterns that that the model has seen 
                
        
        print("err:",err)
        
        
    def test_sample(self, test):
        
        test = test/255 #normalizing input image
        
        yi = np.zeros((20, 1))
        yf = np.zeros((10, 1))

        for i in range(self.h):
        
            weights = np.row_stack(self.i_to_h_weights[i, :])
            yi[i, 0] = self.feed_forward(test, weights)
        
        for i in range(self.out):
        
            weights = np.row_stack(self.h_to_o_weights[i, :])
            yf[i, 0] = self.feed_forward(yi, weights)
        
        print("y :", yf)
        
        #max_vote = max(yf)
        #number_prediction = yf.index(max_vote)
        number_prediction = np.argmax(yf)
        
        return number_prediction
        
    

def sig_fn(z):  #the Sigmoid Function(which will be the activation function that we take)
    
    y = 1/(1 + np.exp(-z))
    
    return(y)

In [99]:
nn = Net() #making an object of network

In [100]:
#start training the network; stop manually when you're happy with the error
nn.batch_update() #training the network

pattern err: [0.12417231]
pattern err: [0.10884218]
pattern err: [0.09816798]
pattern err: [0.07458832]
pattern err: [0.0731893]
pattern err: [0.06132924]
pattern err: [0.04735957]
pattern err: [0.05778977]
pattern err: [0.03954202]
pattern err: [0.05482052]
pattern err: [0.04899832]
pattern err: [0.05221818]
pattern err: [0.04149144]
pattern err: [0.0518624]
pattern err: [0.04420641]
pattern err: [0.05109273]
pattern err: [0.04812639]
pattern err: [0.04892507]
pattern err: [0.04714768]
pattern err: [0.04829776]
pattern err: [0.0457596]
pattern err: [0.04737585]
pattern err: [0.04540338]
pattern err: [0.04433402]
pattern err: [0.03886196]
pattern err: [0.04898373]
pattern err: [0.04668743]
pattern err: [0.04747798]
pattern err: [0.04504487]
pattern err: [0.04906896]
pattern err: [0.04505737]
pattern err: [0.04853102]
pattern err: [0.04790893]
pattern err: [0.04604304]
pattern err: [0.04765257]
pattern err: [0.04866856]
pattern err: [0.04493366]
pattern err: [0.04572772]
pattern err: [0

pattern err: [0.05294994]
pattern err: [0.04409028]
pattern err: [0.03154738]
pattern err: [0.0477407]
pattern err: [0.04320428]
pattern err: [0.05118742]
pattern err: [0.04599871]
pattern err: [0.04459224]
pattern err: [0.02133471]
pattern err: [0.04650433]
pattern err: [0.04894793]
pattern err: [0.04069401]
pattern err: [0.04372148]
pattern err: [0.04642349]
pattern err: [0.04642102]
pattern err: [0.04971841]
pattern err: [0.04453708]
pattern err: [0.04259076]
pattern err: [0.04264094]
pattern err: [0.04699711]
pattern err: [0.03107328]
pattern err: [0.04957021]
pattern err: [0.04601074]
pattern err: [0.04379104]
pattern err: [0.03837193]
pattern err: [0.04952768]
pattern err: [0.05053725]
pattern err: [0.02634811]
pattern err: [0.0420071]
pattern err: [0.04532752]
pattern err: [0.04961001]
pattern err: [0.0441812]
pattern err: [0.0388404]
pattern err: [0.0202751]
pattern err: [0.0486721]
pattern err: [0.04268745]
pattern err: [0.04688882]
pattern err: [0.0098217]
pattern err: [0.048

pattern err: [0.01979799]
pattern err: [0.01449662]
pattern err: [0.00940221]
pattern err: [0.00560513]
pattern err: [0.02183481]
pattern err: [0.06641455]
pattern err: [0.0421943]
pattern err: [0.04959197]
pattern err: [0.05580535]
pattern err: [0.05359119]
pattern err: [0.04427302]
pattern err: [0.04087372]
pattern err: [0.02053082]
pattern err: [0.04501495]
pattern err: [0.04525842]
pattern err: [0.03058973]
pattern err: [0.04771639]
pattern err: [0.04464137]
pattern err: [0.02223076]
pattern err: [0.03869397]
pattern err: [0.01180441]
pattern err: [0.03360615]
pattern err: [0.04882072]
pattern err: [0.04898196]
pattern err: [0.04002512]
pattern err: [0.03472296]
pattern err: [0.00830008]
pattern err: [0.01021665]
pattern err: [0.04935872]
pattern err: [0.0409843]
pattern err: [0.01264639]
pattern err: [0.00603424]
pattern err: [0.00435398]
pattern err: [0.00268466]
pattern err: [0.07891375]
pattern err: [0.01193934]
pattern err: [0.03802835]
pattern err: [0.04234191]
pattern err: [

pattern err: [0.00672974]
pattern err: [0.03044973]
pattern err: [0.04745031]
pattern err: [0.00603068]
pattern err: [0.03087441]
pattern err: [0.00734864]
pattern err: [0.04096422]
pattern err: [0.00589242]
pattern err: [0.00321949]
pattern err: [0.04950408]
pattern err: [0.03599728]
pattern err: [0.00848868]
pattern err: [0.05277739]
pattern err: [0.01174464]
pattern err: [0.01383726]
pattern err: [0.00304586]
pattern err: [0.03317349]
pattern err: [0.00443534]
pattern err: [0.0694376]
pattern err: [0.029143]
pattern err: [0.05150323]
pattern err: [0.03981047]
pattern err: [0.04563726]
pattern err: [0.02267372]
pattern err: [0.02880824]
pattern err: [0.00984148]
pattern err: [0.04615291]
pattern err: [0.00866009]
pattern err: [0.00418886]
pattern err: [0.04136925]
pattern err: [0.00627837]
pattern err: [0.04512904]
pattern err: [0.02719497]
pattern err: [0.01872658]
pattern err: [0.02786196]
pattern err: [0.03627212]
pattern err: [0.00338759]
pattern err: [0.02937629]
pattern err: [0

pattern err: [0.06400911]
pattern err: [0.00774754]
pattern err: [0.04376129]
pattern err: [0.01130455]
pattern err: [0.04865653]
pattern err: [0.00434023]
pattern err: [0.06043106]
pattern err: [0.04327173]
pattern err: [0.00241535]
pattern err: [0.01600894]
pattern err: [0.05423351]
pattern err: [0.05670317]
pattern err: [0.03615737]
pattern err: [0.04133391]
pattern err: [0.02991913]
pattern err: [0.01682047]
pattern err: [0.03791423]
pattern err: [0.05400153]
pattern err: [0.04604235]
pattern err: [0.02080215]
pattern err: [0.00507291]
pattern err: [0.00221098]
pattern err: [0.04317426]
pattern err: [0.02770382]
pattern err: [0.04459723]
pattern err: [0.01981287]
pattern err: [0.04724628]
pattern err: [0.03349575]
pattern err: [0.0210694]
pattern err: [0.04067488]
pattern err: [0.00888135]
pattern err: [0.02385214]
pattern err: [0.01946894]
pattern err: [0.02734249]
pattern err: [0.00894589]
pattern err: [0.01506406]
pattern err: [0.02885139]
pattern err: [0.00700694]
pattern err: 

pattern err: [0.00923294]
pattern err: [0.03164831]
pattern err: [0.06523655]
pattern err: [0.04588723]
pattern err: [0.02074852]
pattern err: [0.01087346]
pattern err: [0.00541725]
pattern err: [0.01149484]
pattern err: [0.05330958]
pattern err: [0.01135607]
pattern err: [0.04245389]
pattern err: [0.00395308]
pattern err: [0.00949329]
pattern err: [0.0087421]
pattern err: [0.03387794]
pattern err: [0.0011595]
pattern err: [0.00107728]
pattern err: [0.00152865]
pattern err: [0.06896335]
pattern err: [0.00781565]
pattern err: [0.02097514]
pattern err: [0.0162742]
pattern err: [0.00984186]
pattern err: [0.01018929]
pattern err: [0.02613158]
pattern err: [0.04762945]
pattern err: [0.05546694]
pattern err: [0.00982305]
pattern err: [0.05680852]
pattern err: [0.00337336]
pattern err: [0.07844501]
pattern err: [0.03833121]
pattern err: [0.04514484]
pattern err: [0.01523057]
pattern err: [0.00758644]
pattern err: [0.0017819]
pattern err: [0.00335804]
pattern err: [0.00431111]
pattern err: [0.

pattern err: [0.02247455]
pattern err: [0.00562611]
pattern err: [0.00169662]
pattern err: [0.00527246]
pattern err: [0.01760105]
pattern err: [0.05897041]
pattern err: [0.04544543]
pattern err: [0.00350205]
pattern err: [0.01036928]
pattern err: [0.00143783]
pattern err: [0.00169335]
pattern err: [0.00828635]
pattern err: [0.03150377]
pattern err: [0.01164116]
pattern err: [0.02706192]
pattern err: [0.00571272]
pattern err: [0.04071278]
pattern err: [0.01479847]
pattern err: [0.00241633]
pattern err: [0.00113131]
pattern err: [0.0557613]
pattern err: [0.07348802]
pattern err: [0.00495673]
pattern err: [0.00865942]
pattern err: [0.01533126]
pattern err: [0.0012227]
pattern err: [0.00193838]
pattern err: [0.07324224]
pattern err: [0.00181583]
pattern err: [0.02527879]
pattern err: [0.00669227]
pattern err: [0.01269033]
pattern err: [0.01174353]
pattern err: [0.00206946]
pattern err: [0.00132149]
pattern err: [0.0363514]
pattern err: [0.02038]
pattern err: [0.00215269]
pattern err: [0.00

pattern err: [0.00611322]
pattern err: [0.0003085]
pattern err: [0.00452849]
pattern err: [0.0005927]
pattern err: [0.00471596]
pattern err: [0.03165161]
pattern err: [0.02383015]
pattern err: [0.00277832]
pattern err: [0.02643393]
pattern err: [0.00127008]
pattern err: [0.03441594]
pattern err: [0.01200783]
pattern err: [0.0022219]
pattern err: [0.01350665]
pattern err: [0.00463197]
pattern err: [0.00357012]
pattern err: [0.02960971]
pattern err: [0.00435392]
pattern err: [0.03177918]
pattern err: [0.00762842]
pattern err: [0.00240002]
pattern err: [0.03577785]
pattern err: [0.04909614]
pattern err: [0.00144796]
pattern err: [0.03783154]
pattern err: [0.00312916]
pattern err: [0.03447007]
pattern err: [0.00147291]
pattern err: [0.00530064]
pattern err: [0.00153228]
pattern err: [0.00266716]
pattern err: [0.00147557]
pattern err: [0.03676059]
pattern err: [0.00224957]
pattern err: [0.00373364]
pattern err: [0.00324768]
pattern err: [0.00563868]
pattern err: [0.0079037]
pattern err: [0.

KeyboardInterrupt: 

I am able to get upto 96% accuracy (4% error)

Testing our trained network with a random test sample..

In [101]:
rand_samp = random.randint(0,10000) #'random' index number to pick up from the test set 

tst_input = np.row_stack(test_img[rand_samp])
ans = nn.test_sample(tst_input)  #testing the network on one test sample
print("The image shows the number : ", ans)

y : [[0.88092007]
 [0.00211449]
 [0.433278  ]
 [0.08025702]
 [0.00406027]
 [0.11943474]
 [0.0053755 ]
 [0.01128393]
 [0.0515695 ]
 [0.00614327]]
The image shows the number :  0


And the actual image is...

In [102]:
print(mndata.display(test_img[rand_samp]))


............................
............................
............................
............................
..........@@................
........@@@@@@..............
.......@@....@@.............
.......@......@@@@..........
......@@........@@@.........
......@...........@@........
......@............@@.......
......@............@@.......
......@.............@@......
......@..............@@.....
......@..............@@.....
......@..............@@.....
......@@..............@@....
......@@..............@@....
.......@..............@.....
.......@@@...........@@.....
........@@@.........@@@.....
.........@@@@.....@@@@......
...........@@@@@@@@@........
..............@@@...........
............................
............................
............................
............................
